In [1]:
import os
import sys
import pickle
import numpy as np
from numpy import gradient
from math import pi
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [2]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'stimuli_out':[], 'w_goal': 0.4, 'w_obst': 0.2, 'ps_trial':[], 'ps_subj':[], 'time_stamp':[], 'stimuli_match':[],
      'pass_order':[], 'p_subj':[], 'v_subj':[], 'v_obst':[], 'a_subj':[]}
data = Data(Hz=90, header=header, info=info)
n_subj = 11
for subj_id in range(n_subj):
    # Import output
    file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1',
                                             'Subj' + str(subj_id).zfill(2) + '.csv'))
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while head + 1 < len(trials):
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        trial_id = int(trials[head+1][1])
        time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
        angle = float(trials[head+1][3])
        speed = float(trials[head+1][5])
        subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
        p0 = Data.filter(subj[:,[0,2]], time_stamp, 90, 4, 0.6)
        v0 = gradient(p0, axis=0) * data.Hz
        a0 = gradient(v0, axis=0) * data.Hz
        goal = np.array([[float(line[-2]), float(line[-1])] for line in trials[head+3:tail+1]])
        obst = np.array([[float(line[-4]), float(line[-3])] for line in trials[head+3:tail+1]])
        if speed:
            for i in range(len(obst) - 1):
                if obst[i][0] != obst[i + 1][0] or obst[i][1] != obst[i + 1][1]:
                    onset = i
                    break
            obst[onset:] = np.linspace(obst[onset], obst[-1], len(obst[onset:]))
            for i in range(onset, len(subj) - 1):
                _beta = beta(p0[i], obst[i], v0[i])
                if abs(_beta) > pi / 2:
                    stimuli_out = i
                    # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
                    pass_order = np.sign(_beta * -angle)
                    break
            v1 = (obst[-1] - obst[onset]) / (len(obst) - 1 - onset) * data.Hz
            stimuli_match = onset
            for i in range(onset, stimuli_out):
                # The first moment when pass_order == order_pred
#                     order_pred = np.sign(min_sep(p0[i], obst[i], v0[i], v1)[0])
                order_pred = np.sign(beta(p0[i], obst[i], v0[i]) * d_psi(p0[i], obst[i], v0[i], v1))
                if pass_order * order_pred > 0:
                    stimuli_match = i
                    break
        else:
            pass_order = 0  # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
            onset = stimuli_out = stimuli_match = None
        data.add_traj(subj)
        ps_trial = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,[0,2]], data.Hz))
        data.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':speed, 'subj_id':subj_id, 
                       'trial_id':trial_id, 'stimuli_onset':onset, 'stimuli_out':stimuli_out, 'time_stamp':time_stamp, 
                       'ps_trial':ps_trial, 'ps_subj':0, 'pass_order':pass_order, 'stimuli_match':stimuli_match,
                       'p_subj':p0, 'v_subj':v0, 'a_subj':a0, 'v_obst':v1})
        head = tail + 1
ps_subj = [0] * n_subj
for i in range(len(data.trajs)):
    if data.info['obst_speed'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)  

In [ ]:
data.info['stimuli_out']

In [ ]:
'''Check different start time'''
for i in range(100):
    t0 = data.info['stimuli_onset'][i]
    t0_match = data.info['stimuli_match'][i]
    t1 = data.info['stimuli_out'][i]
    if t0 != t0_match:
        print(t0, t0_match, t1)

In [ ]:
'''Check matching rate'''
for i in range(1400):
    if not data.info['obst_speed'][i]:
        continue
    pass_order = data.info['pass_order'][i]
    t0 = data.info['stimuli_onset'][i]
    t0_match = data.info['stimuli_match'][i]    
    p0 = data.info['p_subj'][i][t0_match]
    v0 = data.info['v_subj'][i][t0_match]
    p1 = data.info['p_obst'][i][t0_match]
    v1 = data.info['v_obst'][i]
    order_pred = np.sign(beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))
    if pass_order * order_pred < 0:
        print('subj ', data.info['subj_id'][i], 'trial ', data.info['trial_id'][i], 
              'angle ', data.info['obst_angle'][i],
              pass_order, order_pred, beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))